## Step 1: which *Riboviria* use non-standard genetic code?

**Disclaimer**: Following code was used to extract families with non-standard genetic codes and at random add 15 extra families from an as broad taxonomic range as possible. They have been later on manually curated, so this does not reflect the final models!

In [2]:
# First pass: build the taxonomy tree and store genetic codes
tax_to_parent = {}
tax_to_gencode = {}
tax_to_rank = {}

with open("nodes.dmp", "r") as f:
    for line in f:
        parts = [p.strip() for p in line.split("|")]
        tax_id = int(parts[0])
        parent_tax_id = int(parts[1])
        rank = parts[2]
        gencode = int(parts[6])
        
        tax_to_parent[tax_id] = parent_tax_id
        tax_to_gencode[tax_id] = gencode
        tax_to_rank[tax_id] = rank

# Build tax_id to scientific name mapping
tax_to_name = {}
with open("names.dmp", "r") as f:
    for line in f:
        parts = [p.strip() for p in line.split("|")]
        if parts[3] == "scientific name":
            tax_to_name[int(parts[0])] = parts[1]

# Find Riboviria tax_id
riboviria_tax_id = None
for tax_id, name in tax_to_name.items():
    if name.lower() == "riboviria" and tax_to_rank.get(tax_id) == "realm":
        riboviria_tax_id = tax_id
        break

print(f"Riboviria tax_id: {riboviria_tax_id}")

# Function to check if a taxon is under Riboviria
def is_descendant_of(tax_id, ancestor_id, tax_to_parent):
    current = tax_id
    while current != 1:  # 1 is the root
        if current == ancestor_id:
            return True
        current = tax_to_parent.get(current, 1)
    return False

# Function to get full lineage
def get_lineage(tax_id, tax_to_parent, tax_to_name, tax_to_rank):
    lineage = []
    current = tax_id
    while current != 1:
        name = tax_to_name.get(current, "Unknown")
        rank = tax_to_rank.get(current, "no rank")
        lineage.append(f"{name}")
        current = tax_to_parent.get(current, 1)
    return ";".join(reversed(lineage))

# Function to get family for a taxon
def get_family(tax_id, tax_to_parent, tax_to_rank, tax_to_name):
    current = tax_id
    while current != 1:
        if tax_to_rank.get(current) == "family":
            return tax_to_name.get(current, "Unknown family"), current
        current = tax_to_parent.get(current, 1)
    return "No family", None

# Function to get order for a taxon
def get_order(tax_id, tax_to_parent, tax_to_rank, tax_to_name):
    current = tax_id
    while current != 1:
        if tax_to_rank.get(current) == "order":
            return tax_to_name.get(current, "Unknown order"), current
        current = tax_to_parent.get(current, 1)
    return "No order", None

# Filter taxa: belong to Riboviria AND don't use genetic code 1
riboviria_non_standard = []
families_with_non_standard = set()  # Track families with non-standard genetic codes
skipped_no_family = 0
skipped_non_species = 0  # Track non-species taxa

for tax_id, gencode in tax_to_gencode.items():
    if gencode != 1 and is_descendant_of(tax_id, riboviria_tax_id, tax_to_parent):
        # Only include species-level and "no rank" taxids
        rank = tax_to_rank.get(tax_id, "no rank")
        if rank not in ["species", "no rank"]:
            skipped_non_species += 1
            continue
            
        family_name, family_id = get_family(tax_id, tax_to_parent, tax_to_rank, tax_to_name)
        
        # Skip if no family
        if family_name == "No family":
            skipped_no_family += 1
            continue
        
        families_with_non_standard.add((family_name, family_id))
        
        lineage = get_lineage(tax_id, tax_to_parent, tax_to_name, tax_to_rank)
        riboviria_non_standard.append({
            'tax_id': tax_id,
            'gencode': gencode,
            'family': family_name,
            'family_id': family_id if family_id else 0,
            'lineage': lineage,
            'name': tax_to_name.get(tax_id, "Unknown"),
            'rank': tax_to_rank.get(tax_id, "no rank")
        })

print(f"Found {len(families_with_non_standard)} families with non-standard genetic codes")
print(f"Skipped {skipped_no_family} taxa with no family assignment")
print(f"Skipped {skipped_non_species} non-species taxa (genus, etc.)")

# Add all families (as family-level taxids) that have non-standard genetic code species
families_with_non_standard_list = []
for family_name, family_id in families_with_non_standard:
    if family_id and family_id != 0:
        families_with_non_standard_list.append({
            'tax_id': family_id,
            'gencode': 1,  # Family itself uses genetic code 1, for downloading standard code members
            'family': family_name,
            'family_id': family_id,
            'order': get_order(family_id, tax_to_parent, tax_to_rank, tax_to_name)[0],
            'lineage': get_lineage(family_id, tax_to_parent, tax_to_name, tax_to_rank),
            'name': family_name,
            'rank': 'family'
        })

# Find Riboviria families with genetic code 1 (diverse by order)
# Exclude families that already have non-standard genetic code species
riboviria_standard = {}  # key: (order_name, family_name), value: tax_id
for tax_id, gencode in tax_to_gencode.items():
    if gencode == 1 and is_descendant_of(tax_id, riboviria_tax_id, tax_to_parent):
        if tax_to_rank.get(tax_id) == "family":
            family_name = tax_to_name.get(tax_id, "Unknown")
            
            # Skip if this family already has non-standard genetic code species
            if (family_name, tax_id) in families_with_non_standard:
                continue
            
            order_name, order_id = get_order(tax_id, tax_to_parent, tax_to_rank, tax_to_name)
            riboviria_standard[(order_name, family_name)] = tax_id

# Select 15 families with maximum order diversity
from collections import defaultdict
order_to_families = defaultdict(list)
for (order_name, family_name), family_tax_id in riboviria_standard.items():
    order_to_families[order_name].append((family_name, family_tax_id))

# Sort orders by number of families (to prioritize diverse orders)
sorted_orders = sorted(order_to_families.items(), key=lambda x: len(x[1]))

# Select families: one from each order first, then fill remaining slots
selected_families = []
remaining_slots = 15

# First pass: one family per order
for order_name, families in sorted_orders:
    if remaining_slots > 0 and families:
        family_name, family_tax_id = families[0]
        selected_families.append({
            'tax_id': family_tax_id,
            'gencode': 1,
            'family': family_name,
            'family_id': family_tax_id,
            'order': order_name,
            'lineage': get_lineage(family_tax_id, tax_to_parent, tax_to_name, tax_to_rank),
            'name': family_name,
            'rank': 'family'
        })
        remaining_slots -= 1

# Second pass: fill remaining slots with families from orders with most families
if remaining_slots > 0:
    sorted_orders_desc = sorted(order_to_families.items(), key=lambda x: len(x[1]), reverse=True)
    for order_name, families in sorted_orders_desc:
        for family_name, family_tax_id in families[1:]:  # Skip first (already selected)
            if remaining_slots > 0:
                # Check if not already selected
                if family_tax_id not in [f['tax_id'] for f in selected_families]:
                    selected_families.append({
                        'tax_id': family_tax_id,
                        'gencode': 1,
                        'family': family_name,
                        'family_id': family_tax_id,
                        'order': order_name,
                        'lineage': get_lineage(family_tax_id, tax_to_parent, tax_to_name, tax_to_rank),
                        'name': family_name,
                        'rank': 'family'
                    })
                    remaining_slots -= 1
            if remaining_slots == 0:
                break
        if remaining_slots == 0:
            break

# Combine all lists
all_entries = riboviria_non_standard + families_with_non_standard_list + selected_families

# Sort by family name, then by tax_id
all_entries.sort(key=lambda x: (x['family'], x['tax_id']))

# Write to TSV file
with open("riboviria_genetic_codes.tsv", "w") as out:
    out.write("family\ttaxid\trank\tscientific_name\tgenetic_code\tlineage\n")
    for entry in all_entries:
        out.write(f"{entry['family']}\t{entry['tax_id']}\t{entry['rank']}\t{entry['name']}\t{entry['gencode']}\t{entry['lineage']}\n")

print(f"\nFound {len(riboviria_non_standard)} Riboviria taxa with non-standard genetic codes")
print(f"Added {len(families_with_non_standard_list)} families with non-standard genetic code species (for gc=1 downloads)")
print(f"Added {len(selected_families)} additional Riboviria families with genetic code 1 (diverse orders)")
print(f"Total: {len(all_entries)} entries")
print(f"Output written to riboviria_genetic_codes.tsv")

# Print summary
from collections import Counter
family_counts = Counter(entry['family'] for entry in all_entries)
gencode_counts = Counter(entry['gencode'] for entry in all_entries)

print(f"\nBreakdown by genetic code:")
for gencode, count in gencode_counts.most_common():
    print(f"  {gencode}: {count}")

print(f"\nFamilies with non-standard genetic code species:")
for family_name, family_id in sorted(families_with_non_standard):
    print(f"  {family_name}")

print(f"\nAdditional genetic code 1 families by order:")
order_counts = Counter(entry.get('order', 'N/A') for entry in selected_families)
for order_name, count in order_counts.most_common():
    print(f"  {order_name}: {count}")

print(f"\nAdditional genetic code 1 families:")
for entry in selected_families:
    print(f"  {entry['family']}")

Riboviria tax_id: 2559587
Found 17 families with non-standard genetic codes
Skipped 1026 taxa with no family assignment
Skipped 535 non-species taxa (genus, etc.)

Found 4563 Riboviria taxa with non-standard genetic codes
Added 17 families with non-standard genetic code species (for gc=1 downloads)
Added 15 additional Riboviria families with genetic code 1 (diverse orders)
Total: 4595 entries
Output written to riboviria_genetic_codes.tsv

Breakdown by genetic code:
  11: 2774
  4: 1763
  1: 32
  6: 8
  3: 6
  16: 5
  5: 4
  14: 3

Families with non-standard genetic code species:
  Atkinsviridae
  Blumeviridae
  Botourmiaviridae
  Cystoviridae
  Dicistroviridae
  Duinviridae
  Fiersviridae
  Mitoviridae
  Narnaviridae
  Orthototiviridae
  Partitiviridae
  Picobirnaviridae
  Picornaviridae
  Qinviridae
  Solspiviridae
  Steitzviridae
  Tombusviridae

Additional genetic code 1 families by order:
  Blubervirales: 1
  Amarillovirales: 1
  Patatavirales: 1
  Stellavirales: 1
  Naedrevirales:

## Step 2: Extract taxids of selected *Riboviria* families

In [6]:
import polars as pl
from pathlib import Path

def load_taxonomy():
    """Load taxonomy tree from nodes.dmp"""
    tax_to_parent = {}
    tax_to_gencode = {}
    tax_to_rank = {}
    
    with open("nodes.dmp", "r") as f:
        for line in f:
            parts = [p.strip() for p in line.split("|")]
            tax_id = int(parts[0])
            parent_tax_id = int(parts[1])
            rank = parts[2]
            gencode = int(parts[6])
            
            tax_to_parent[tax_id] = parent_tax_id
            tax_to_gencode[tax_id] = gencode
            tax_to_rank[tax_id] = rank
    
    return tax_to_parent, tax_to_gencode, tax_to_rank

def get_all_descendants(tax_id, tax_to_parent):
    """Get all descendant taxids of a given taxid"""
    # Build parent-to-children mapping
    children = {}
    for child, parent in tax_to_parent.items():
        if parent not in children:
            children[parent] = []
        children[parent].append(child)
    
    # Recursively get all descendants
    descendants = set()
    def get_descendants_recursive(tid):
        descendants.add(tid)
        if tid in children:
            for child in children[tid]:
                get_descendants_recursive(child)
    
    get_descendants_recursive(tax_id)
    return descendants

def main():
    # Load taxonomy
    print("Loading taxonomy...")
    tax_to_parent, tax_to_gencode, tax_to_rank = load_taxonomy()
    print("Taxonomy loaded\n")
    
    # Read the TSV
    df = pl.read_csv("riboviria_genetic_codes.tsv", separator="\t")
    
    # Create output directory
    output_dir = Path("taxid_files")
    output_dir.mkdir(exist_ok=True)
    
    # Find families with multiple genetic codes
    families_with_multiple_gencodes = set(
        df.group_by('family')
          .agg(pl.col('genetic_code').n_unique().alias('gencode_count'))
          .filter(pl.col('gencode_count') > 1)['family']
          .to_list()
    )
    
    print(f"Families with multiple genetic codes: {len(families_with_multiple_gencodes)}")
    
    # Group by family and genetic code
    for (family_name, genetic_code), group_df in df.group_by(['family', 'genetic_code']):
        output_file = output_dir / f"{family_name}_{genetic_code}.txt"
        
        # Get family taxid (should only be one per family-genetic_code combo)
        family_taxid = group_df.filter(pl.col('rank') == 'family')['taxid'].to_list()
        
        if family_taxid and family_name in families_with_multiple_gencodes:
            # Family with multiple genetic codes: get all descendants and filter by genetic code
            all_descendants = get_all_descendants(family_taxid[0], tax_to_parent)
            taxids = [
                tid for tid in all_descendants 
                if tax_to_gencode.get(tid, 1) == genetic_code 
                and tax_to_rank.get(tid, "no rank") in ["species", "no rank"]
            ]
        elif family_taxid:
            # Single genetic code family: get descendants and filter by rank
            all_descendants = get_all_descendants(family_taxid[0], tax_to_parent)
            taxids = [
                tid for tid in all_descendants 
                if tax_to_rank.get(tid, "no rank") in ["species", "no rank"]
            ]
        else:
            # Non-family entries: filter by rank
            taxids = [
                tid for tid in group_df['taxid'].to_list()
                if tax_to_rank.get(tid, "no rank") in ["species", "no rank"]
            ]
        
        # Write to file
        taxids = sorted(set(taxids))
        with open(output_file, 'w') as f:
            for taxid in taxids:
                f.write(f"{taxid}\n")
        
        print(f"Saved {len(taxids)} taxids to {output_file}")
    
    # Delete empty files
    print("\nCleaning up empty files...")
    deleted_count = 0
    for file in output_dir.glob("*.txt"):
        if file.stat().st_size == 0:
            file.unlink()
            print(f"Deleted empty file: {file.name}")
            deleted_count += 1
    
    if deleted_count > 0:
        print(f"Deleted {deleted_count} empty file(s)")
    else:
        print("No empty files found")

if __name__ == "__main__":
    main()

Loading taxonomy...


Taxonomy loaded

Families with multiple genetic codes: 17
Saved 50 taxids to taxid_files/Qinviridae_1.txt
Saved 605 taxids to taxid_files/Dicistroviridae_1.txt
Saved 1 taxids to taxid_files/Partitiviridae_4.txt
Saved 13 taxids to taxid_files/Betaormycoviridae_1.txt
Saved 1538 taxids to taxid_files/Flaviviridae_1.txt
Saved 1 taxids to taxid_files/Picobirnaviridae_14.txt
Saved 3 taxids to taxid_files/Narnaviridae_4.txt
Saved 1 taxids to taxid_files/Picornaviridae_6.txt
Saved 1059 taxids to taxid_files/Partitiviridae_1.txt
Saved 0 taxids to taxid_files/Solspiviridae_1.txt
Saved 759 taxids to taxid_files/Orthototiviridae_1.txt
Saved 1828 taxids to taxid_files/Sedoreoviridae_1.txt
Saved 114 taxids to taxid_files/Solspiviridae_11.txt
Saved 15 taxids to taxid_files/Mitoviridae_1.txt
Saved 21 taxids to taxid_files/Barnaviridae_1.txt
Saved 2 taxids to taxid_files/Narnaviridae_5.txt
Saved 2 taxids to taxid_files/Picobirnaviridae_4.txt
Saved 1 taxids to taxid_files/Botourmiaviridae_16.txt
Saved 1

---

The final models have been processed with following code in CLI scripts from this point:

## Step 3: Download genomes with NCBI datasets

In [1]:
from pathlib import Path
import subprocess
import shutil
import zipfile

def download_and_merge_genomes(taxid_file, output_fasta):
    """Download genomes using --inputfile and merge into single FASTA"""
    taxid_file = Path(taxid_file)
    
    if not taxid_file.exists():
        print(f"Warning: {taxid_file} does not exist, skipping...")
        return
    
    print(f"Downloading genomes for {taxid_file.name}...")
    
    # Read taxids
    with open(taxid_file, 'r') as f:
        taxids = [line.strip() for line in f if line.strip()]
    
    if not taxids:
        print(f"  No taxids to download")
        return
    
    print(f"  Found {len(taxids)} taxids to download")
    
    # Split into batches of 100
    batch_size = 100
    batches = [taxids[i:i + batch_size] for i in range(0, len(taxids), batch_size)]
    
    print(f"  Processing {len(batches)} batch(es)...")
    
    temp_base = Path(f"temp_{taxid_file.stem}")
    temp_base.mkdir(exist_ok=True)
    
    try:
        # Download all batches
        for batch_num, batch in enumerate(batches, 1):
            print(f"  Batch {batch_num}/{len(batches)} ({len(batch)} taxids)...")
            
            batch_file = temp_base / f"batch_{batch_num}.txt"
            with open(batch_file, 'w') as f:
                for taxid in batch:
                    f.write(f"{taxid}\n")
            
            zip_file = temp_base / f"batch_{batch_num}.zip"
            cmd = [
                "datasets", "download", "virus", "genome", "taxon",
                "--inputfile", str(batch_file),
                "--filename", str(zip_file)
            ]
            
            result = subprocess.run(cmd, capture_output=True, text=True)
            
            if result.returncode == 0 and zip_file.exists():
                batch_dir = temp_base / f"batch_{batch_num}"
                with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                    zip_ref.extractall(batch_dir)
                zip_file.unlink()
            else:
                print(f"    Warning: Failed to download batch {batch_num}")
                if result.stderr:
                    print(f"    Error: {result.stderr}")
            
            batch_file.unlink()
        
        # Merge all FASTA files from all batches
        with open(output_fasta, 'w') as outfile:
            for fasta_file in sorted(temp_base.rglob("*.fna")):
                with open(fasta_file, 'r') as infile:
                    outfile.write(infile.read())
        
        print(f"  Saved to {output_fasta}")
    
    finally:
        if temp_base.exists():
            shutil.rmtree(temp_base)

def main():
    output_dir = Path("genomes_output")
    output_dir.mkdir(exist_ok=True)
    
    taxid_dir = Path("taxid_files")
    
    for taxid_file in sorted(taxid_dir.glob("*.txt")):
        output_fasta = output_dir / f"{taxid_file.stem}.fasta"
        download_and_merge_genomes(taxid_file, output_fasta)
        print()

if __name__ == "__main__":
    main()

  Found 1 taxids to download
  Processing 1 batch(es)...
  Batch 1/1 (1 taxids)...
  Saved to genomes_output/Alphaormycoviridae_1.fasta

  Found 1 taxids to download
  Processing 1 batch(es)...
  Batch 1/1 (1 taxids)...
  Saved to genomes_output/Aspiviridae_1.fasta

  Found 1 taxids to download
  Processing 1 batch(es)...
  Batch 1/1 (1 taxids)...
  Saved to genomes_output/Astroviridae_1.fasta

  No taxids to download

  Found 306 taxids to download
  Processing 4 batch(es)...
  Batch 1/4 (100 taxids)...
  Batch 2/4 (100 taxids)...
  Batch 3/4 (100 taxids)...
  Batch 4/4 (6 taxids)...
  Saved to genomes_output/Atkinsviridae_11.fasta

  Found 1 taxids to download
  Processing 1 batch(es)...
  Batch 1/1 (1 taxids)...
  Saved to genomes_output/Barnaviridae_1.fasta

  Found 1 taxids to download
  Processing 1 batch(es)...
  Batch 1/1 (1 taxids)...
  Saved to genomes_output/Betaormycoviridae_1.fasta

  No taxids to download

  Found 124 taxids to download
  Processing 2 batch(es)...
  Batch

: 

## Step 4: Cluster genomes
Cluster the genomes per Family with `mmseqs easy-cluster` on 80% ANI and 80% coverage.

In [1]:
from pathlib import Path
import subprocess
import shutil

def cluster_genomes(input_fasta, output_fasta, identity=0.8, coverage=0.8):
    """Cluster sequences using MMseqs2 easy-cluster and extract representatives"""
    input_fasta = Path(input_fasta)
    
    if not input_fasta.exists():
        print(f"Warning: {input_fasta} does not exist, skipping...")
        return False
    
    print(f"Clustering {input_fasta.name} at {identity*100}% identity...")
    
    # Create temporary directory for MMseqs2 output
    temp_dir = Path(f"temp_mmseqs_{input_fasta.stem}")
    temp_dir.mkdir(exist_ok=True)
    
    try:
        # Run MMseqs2 easy-cluster
        cluster_prefix = temp_dir / "cluster"
        cmd = [
            "mmseqs", "easy-cluster",
            str(input_fasta),
            str(cluster_prefix),
            str(temp_dir / "tmp"),
            "--min-seq-id", str(identity),
            "--threads", str(36),
            "-c", str(coverage),
            "--cov-mode", "0"  # coverage of query and target
        ]
        
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        if result.returncode != 0:
            print(f"  Error running MMseqs2: {result.stderr}")
            return False
        
        # Get representative sequences file
        rep_seq_file = temp_dir / "cluster_rep_seq.fasta"
        
        if not rep_seq_file.exists():
            print(f"  Error: Representative sequences file not found")
            return False
        
        # Copy representative sequences to output
        shutil.copy(rep_seq_file, output_fasta)
        
        # Count sequences
        with open(input_fasta, 'r') as f:
            original_count = sum(1 for line in f if line.startswith('>'))
        with open(output_fasta, 'r') as f:
            clustered_count = sum(1 for line in f if line.startswith('>'))
        
        print(f"  Clustered {original_count} sequences to {clustered_count} representatives")
        
        return True
        
    finally:
        # Clean up temporary directory
        if temp_dir.exists():
            shutil.rmtree(temp_dir)

def main():
    genomes_dir = Path("genomes_output")
    clustered_dir = Path("genomes_clustered")
    clustered_dir.mkdir(exist_ok=True)
    
    # Cluster all genome files
    for fasta_file in sorted(genomes_dir.glob("*.fasta")):
        output_fasta = clustered_dir / fasta_file.name
        cluster_genomes(fasta_file, output_fasta, identity=0.8, coverage=0.8)
        print()

if __name__ == "__main__":
    main()

Clustering Alphaormycoviridae_1.fasta at 80.0% identity...


  Clustered 17 sequences to 17 representatives

Clustering Aspiviridae_1.fasta at 80.0% identity...
  Clustered 1028 sequences to 223 representatives

Clustering Astroviridae_1.fasta at 80.0% identity...
  Clustered 16084 sequences to 3250 representatives

Clustering Atkinsviridae_11.fasta at 80.0% identity...
  Clustered 478 sequences to 119 representatives

Clustering Barnaviridae_1.fasta at 80.0% identity...
  Clustered 69 sequences to 66 representatives

Clustering Betaormycoviridae_1.fasta at 80.0% identity...
  Clustered 11 sequences to 11 representatives

Clustering Blumeviridae_11.fasta at 80.0% identity...
  Clustered 109 sequences to 44 representatives

Clustering Botourmiaviridae_1.fasta at 80.0% identity...
  Clustered 3800 sequences to 1024 representatives

Clustering Botourmiaviridae_16.fasta at 80.0% identity...
  Clustered 1 sequences to 1 representatives

Clustering Botourmiaviridae_4.fasta at 80.0% identity...
  Clustered 3 sequences to 2 representatives

Clustering C

## Step 5: Train pyrodigal models on downloaded genomes
- `_MIN_SINGLE_GENOME` variable in pyrodigal was reduced to 5000
- `_IDEAL_SINGLE_GENOME` variable in pyrodigal was reduced to 10000

In [ ]:
import polars as pl
from pathlib import Path
import pyrodigal
import json
import Bio.SeqIO

def train_pyrodigal_model(fasta_file, genetic_code, output_json):
    """Train a Pyrodigal model on a FASTA file with specified genetic code"""
    fasta_file = Path(fasta_file)
    
    if not fasta_file.exists():
        print(f"Warning: {fasta_file} does not exist, skipping...")
        return None
    
    print(f"Training model for {fasta_file.name} (genetic code {genetic_code})...")
    
    # Read sequences from FASTA using Biopython
    records = list(Bio.SeqIO.parse(fasta_file, "fasta"))
    
    if not records:
        print(f"  Warning: No sequences found in {fasta_file}")
        return None
    
    print(f"  Found {len(records)} sequences")
    
    # Calculate total length
    total_bases = sum(len(rec.seq) for rec in records)
    print(f"  Total length: {total_bases} bp")
    
    if total_bases < 5000:
        print(f"  Total length < 5kb, skipping training for this family")
        return None
    
    # Train ORF finder
    try:
        orf_finder = pyrodigal.GeneFinder()
        training_info = orf_finder.train(
            *(bytes(seq.seq) for seq in records), translation_table=genetic_code
        )
        print(f"  Successfully trained model")
    except Exception as e:
        print(f"  Error: Could not train model: {e}")
        return None
    
    # Save training info to JSON
    with open(output_json, 'w') as f:
        json.dump(training_info.to_dict(), f, indent=2)
    
    print(f"  Saved training model to {output_json}")
    
    # Return the training info dict and genetic code for adding to meta.json
    return training_info.to_dict(), genetic_code

def main():
    # Read the original TSV to get genetic codes for each family
    df = pl.read_csv("riboviria_genetic_codes.tsv", separator="\t")
    
    # Get unique family-genetic_code combinations
    family_gc = df.select(['family', 'genetic_code']).unique()
    
    # Create output directory for models
    models_dir = Path("pyrodigal_models")
    models_dir.mkdir(exist_ok=True)
    
    genomes_dir = Path("genomes_clustered")
    
    # Path to meta.json
    meta_json_path = Path("/user/leuven/337/vsc33750/data/software/pyrodigal-gv/pyrodigal_gv/meta.json")
    
    # Read existing meta.json
    existing_models = []
    if meta_json_path.exists():
        try:
            with open(meta_json_path, 'r') as f:
                content = f.read().strip()
                if content:  # Check if file has content
                    existing_models = json.loads(content)
                else:
                    print("meta.json is empty, starting fresh")
        except json.JSONDecodeError:
            print("Warning: meta.json is not valid JSON, starting fresh")
            existing_models = []
    else:
        print("meta.json does not exist, will create new file")
    
    print(f"Existing models in meta.json: {len(existing_models)}")
    
    # Get the highest existing index
    max_index = 0
    for model in existing_models:
        desc = model.get("description", "")
        if "|" in desc:
            try:
                index = int(desc.split("|")[0])
                max_index = max(max_index, index)
            except:
                pass
    
    print(f"Starting new model index at: {max_index + 1}")
    
    # Collect all new models
    new_models = []
    current_index = max_index + 1
    
    # Train model for each family
    for row in family_gc.iter_rows(named=True):
        family = row['family']
        genetic_code = row['genetic_code']
        
        fasta_file = genomes_dir / f"{family}_{genetic_code}.fasta"
        output_json = models_dir / f"{family}_{genetic_code}_model.json"
        
        result = train_pyrodigal_model(fasta_file, genetic_code, output_json)
        
        if result is not None:
            training_info_dict, gc_table = result
            
            # Get GC content and convert to percentage (rounded to 1 decimal)
            gc_content = training_info_dict.get("gc")
            gc_percent = round(gc_content * 100, 1)
            
            # Get uses_sd from training_info
            uses_sd = training_info_dict.get("uses_sd")
            
            # Create description in format: index|model_name|V|gc|translation_table|uses_sd
            description = f"{current_index}|{family}_{genetic_code}_model|V|{gc_percent}|{gc_table}|{uses_sd}"
            
            # Add to new models list
            model_entry = {
                "description": description,
                "training_info": training_info_dict
            }
            new_models.append(model_entry)
            
            print(f"  Model description: {description}")
            
            current_index += 1
        
        print()
    
    print(f"New models to add: {len(new_models)}")
    
    # Combine existing and new models
    all_models = existing_models + new_models
    
    # Write back to meta.json
    with open(meta_json_path, 'w') as f:
        json.dump(all_models, f, indent=2)
    
    print(f"Updated meta.json with {len(all_models)} total models")

if __name__ == "__main__":
    main()

meta.json is empty, starting fresh
Existing models in meta.json: 0
Starting new model index at: 1
Training model for Alphaormycoviridae_1.fasta (genetic code 1)...
  Found 17 sequences
  Total length: 40837 bp
  Successfully trained model
  Saved training model to pyrodigal_models/Alphaormycoviridae_1_model.json
  Model description: 1|Alphaormycoviridae_1_model|V|44.6|1|0


Training model for Fiersviridae_4.fasta (genetic code 4)...
  Found 7 sequences
  Total length: 8981 bp
  Successfully trained model
  Saved training model to pyrodigal_models/Fiersviridae_4_model.json
  Model description: 2|Fiersviridae_4_model|V|49.2|4|1

Training model for Picornaviridae_6.fasta (genetic code 6)...
  Found 8 sequences
  Total length: 13014 bp
  Successfully trained model
  Saved training model to pyrodigal_models/Picornaviridae_6_model.json
  Model description: 3|Picornaviridae_6_model|V|43.0|6|0

Training model for Aspiviridae_1.fasta (genetic code 1)...
  Found 223 sequences
  Total length: 634

/tmp/ipykernel_569362/2054383199.py:37: UserWarning: sequence should be at least 10000 characters (9065 found)
  training_info = orf_finder.train(


  Successfully trained model
  Saved training model to pyrodigal_models/Aspiviridae_1_model.json
  Model description: 4|Aspiviridae_1_model|V|36.0|1|0




Training model for Mitoviridae_1.fasta (genetic code 1)...
  Found 15 sequences
  Total length: 33378 bp
  Successfully trained model
  Saved training model to pyrodigal_models/Mitoviridae_1_model.json
  Model description: 5|Mitoviridae_1_model|V|43.9|1|0

Training model for Narnaviridae_1.fasta (genetic code 1)...
  Found 1376 sequences
  Total length: 3224022 bp
  Successfully trained model
  Saved training model to pyrodigal_models/Narnaviridae_1_model.json
  Model description: 6|Narnaviridae_1_model|V|50.5|1|0

Training model for Flaviviridae_1.fasta (genetic code 1)...
  Found 39228 sequences
  Total length: 25761724 bp
  Successfully trained model
  Saved training model to pyrodigal_models/Flaviviridae_1_model.json
  Model description: 7|Flaviviridae_1_model|V|48.5|1|0

Training model for Yadokariviridae_1.fasta (genetic code 1

/tmp/ipykernel_569362/2054383199.py:37: UserWarning: sequence should be at least 10000 characters (8021 found)
  training_info = orf_finder.train(
/tmp/ipykernel_569362/2054383199.py:37: UserWarning: sequence should be at least 10000 characters (6811 found)
  training_info = orf_finder.train(
/tmp/ipykernel_569362/2054383199.py:37: UserWarning: sequence should be at least 10000 characters (5733 found)
  training_info = orf_finder.train(


  Saved training model to pyrodigal_models/Tombusviridae_4_model.json
  Model description: 10|Tombusviridae_4_model|V|48.4|4|0

Training model for Narnaviridae_6.fasta (genetic code 6)...
  Found 2 sequences
  Total length: 6787 bp
  Successfully trained model
  Saved training model to pyrodigal_models/Narnaviridae_6_model.json
  Model description: 11|Narnaviridae_6_model|V|51.3|6|0

Training model for Botourmiaviridae_4.fasta (genetic code 4)...
  Found 2 sequences
  Total length: 5709 bp
  Successfully trained model
  Saved training model to pyrodigal_models/Botourmiaviridae_4_model.json
  Model description: 12|Botourmiaviridae_4_model|V|44.8|4|1

Training model for Duinviridae_11.fasta (genetic code 11)...
  Found 14 sequences
  Total length: 47647 bp
  Successfully trained model
  Saved training model to pyrodigal_models/Duinviridae_11_model.json
  Model description: 13|Duinviridae_11_model|V|43.6|11|1

Training model for Orthomyxoviridae_1.fasta (genetic code 1)...
  Found 27755 s

...

  Successfully trained model
  Saved training model to pyrodigal_models/Tombusviridae_16_model.json
  Model description: 37|Tombusviridae_16_model|V|53.1|16|1

Training model for Blumeviridae_11.fasta (genetic code 11)...
  Found 44 sequences
  Total length: 185304 bp
  Successfully trained model
  Saved training model to pyrodigal_models/Blumeviridae_11_model.json
  Model description: 38|Blumeviridae_11_model|V|45.2|11|1

Training model for Hepadnaviridae_1.fasta (genetic code 1)...
  Found 12657 sequences
  Total length: 5014054 bp
  Successfully trained model
  Saved training model to pyrodigal_models/Hepadnaviridae_1_model.json
  Model description: 39|Hepadnaviridae_1_model|V|47.2|1|0

Training model for Picobirnaviridae_1.fasta (genetic code 1)...
  Found 4041 sequences
  Total length: 6362947 bp
  Successfully trained model
  Saved training model to pyrodigal_models/Picobirnaviridae_1_model.json
  Model description: 40|Picobirnaviridae_1_model|V|42.0|1|1

Training model for Sedor

/tmp/ipykernel_569362/2054383199.py:37: UserWarning: sequence should be at least 10000 characters (8262 found)
  training_info = orf_finder.train(


  Saved training model to pyrodigal_models/Qinviridae_6_model.json
  Model description: 45|Qinviridae_6_model|V|47.8|6|1

Training model for Potyviridae_1.fasta (genetic code 1)...
  Found 3146 sequences
  Total length: 6884505 bp
  Successfully trained model
  Saved training model to pyrodigal_models/Potyviridae_1_model.json
  Model description: 46|Potyviridae_1_model|V|41.9|1|0


Training model for Mitoviridae_4.fasta (genetic code 4)...
  Found 2947 sequences
  Total length: 6421352 bp
  Successfully trained model
  Saved training model to pyrodigal_models/Mitoviridae_4_model.json
  Model description: 47|Mitoviridae_4_model|V|42.7|4|0


Training model for Astroviridae_1.fasta (genetic code 1)...
  Found 3250 sequences
  Total length: 6450610 bp
  Successfully trained model
  Saved training model to pyrodigal_models/Astroviridae_1_model.json
  Model description: 48|Astroviridae_1_model|V|45.9|1|0

Training model for Cystoviridae_11.fasta (genetic code 11)...
  Found 143 sequences
  T

/tmp/ipykernel_569362/2054383199.py:37: UserWarning: sequence should be at least 10000 characters (9168 found)
  training_info = orf_finder.train(


  Successfully trained model
  Saved training model to pyrodigal_models/Atkinsviridae_11_model.json
  Model description: 51|Atkinsviridae_11_model|V|49.0|11|1


New models to add: 51
Updated meta.json with 51 total models


# Step 6: add extra models for families that didn't perform well
- Fimoviridae: 33 (26.6%): 1980412
- Unknown: 21 (16.9%)
- Polymycoviridae: 21 (16.9%): 2732900
- Pseudototiviridae: 9 (7.3%): 3152147
- Tymoviridae: 7 (5.6%): 249184
- Chrysoviridae: 7 (5.6%): 249310
- Matonaviridae: 4 (3.2%): 2560066
- Flaviviridae: 4 (3.2%)
- Peribunyaviridae: 4 (3.2%): 1980416
- Caulimoviridae: 3 (2.4%): 186534
- Mitoviridae: 2 (1.6%)
- Curvulaviridae: 2 (1.6%)
- Phenuiviridae: 2 (1.6%)
- Mononiviridae: 1 (0.8%)
- Alternaviridae: 1 (0.8%)
- Hantaviridae: 1 (0.8%)
- Xinmoviridae: 1 (0.8%)
- Steitzviridae: 1 (0.8%)

Extra:
- *Mitoviridae* were split into genera + removed models with genetic codes 1 and 3 as ther are very likely genetic code 4
- Split *Flaviviridae* into *Pegivirus*, *Pestivirus*, *Hepacivirus* and *Flaviviridae* containing *Orthoflavivirus*, *Flavivirus* and *unclassified Flaviviridae*
- Added *Spinareoviridae*


Final models from `taxid_files` folder:

In [ ]:
%%bash
# CLI Python scripts have been LLM generated and contain arguments that are not necessary/confusing, only use commands below
python scripts/get_taxid_info.py -i taxid_files -o final_models.tsv
python scripts/download_genomes.py --input-dir taxid_files --output-dir genomes_final_models
python scripts/cluster_genomes.py --input-dir genomes_final_models --output-dir clustered_genomes_final_models -t 10
python scripts/train_models.py --tsv final_models.tsv --meta-json /user/leuven/337/vsc33750/data/software/pyrodigal-rv/pyrodigal_rv/meta.json --genomes-dir clustered_genomes_final_models/

In [8]:
import polars as pl
df = pl.read_csv("final_models.tsv", separator="\t")

df2 = df.select([
    "family",
    pl.when(pl.col("rank").is_in(["family", "genus"]))
      .then(pl.col("rank"))
      .otherwise(pl.lit("NA"))
      .alias("rank"),
    pl.when(pl.col("rank").is_in(["family", "genus"]))
      .then(pl.col("scientific_name"))
      .otherwise(pl.lit("NA"))
      .alias("scientific_name"),
    "genetic_code"
]).unique()

df2 = df2.with_columns([
    pl.when(pl.col("scientific_name") == "NA")
      .then(pl.col("family"))
      .otherwise(pl.col("scientific_name"))
      .alias("scientific_name"),
    pl.when(pl.col("rank") == "NA")
      .then(pl.lit("family"))
      .otherwise(pl.col("rank"))
      .alias("rank"),
    pl.when(pl.col("rank") == "genus")
      .then(pl.col("family"))
      .otherwise(pl.lit("NA"))
      .alias("parent_family")
]).drop("family").sort(["parent_family", "scientific_name"]).select([
    "rank",
    "parent_family",
    "scientific_name",
    "genetic_code"
])

df2.write_csv("final_models_summary.tsv", separator="\t")
print(df2)

shape: (77, 4)
┌────────┬───────────────┬─────────────────┬──────────────┐
│ rank   ┆ parent_family ┆ scientific_name ┆ genetic_code │
│ ---    ┆ ---           ┆ ---             ┆ ---          │
│ str    ┆ str           ┆ str             ┆ i64          │
╞════════╪═══════════════╪═════════════════╪══════════════╡
│ genus  ┆ Flaviviridae  ┆ Hepacivirus     ┆ 1            │
│ genus  ┆ Flaviviridae  ┆ Pegivirus       ┆ 1            │
│ genus  ┆ Flaviviridae  ┆ Pestivirus      ┆ 1            │
│ genus  ┆ Mitoviridae   ┆ Duamitovirus    ┆ 4            │
│ genus  ┆ Mitoviridae   ┆ Kvaramitovirus  ┆ 4            │
│ …      ┆ …             ┆ …               ┆ …            │
│ family ┆ NA            ┆ Tombusviridae   ┆ 6            │
│ family ┆ NA            ┆ Tombusviridae   ┆ 1            │
│ family ┆ NA            ┆ Tymoviridae     ┆ 1            │
│ family ┆ NA            ┆ Yadokariviridae ┆ 1            │
│ family ┆ NA            ┆ Yueviridae      ┆ 1            │
└────────┴───────────────

# Step 7 remove unused models for speed up

In [1]:
%%bash
cd refseq_genomes
cat time.txt

pyrodigal-rv closed

real	329m46.147s
user	282m35.547s
sys	795m29.513s
pyrodigal vanilla closed

real	12m15.913s
user	10m19.463s
sys	29m7.417s
pyrodigal-rv open

real	339m48.647s
user	290m24.951s
sys	820m18.577s
pyrodigal-rv closed start

real	339m16.480s
user	289m27.950s
sys	821m29.296s
pyrodigal vanilla open

real	12m26.421s
user	10m24.134s
sys	29m23.769s
pyrodigal vanilla closed start

real	12m15.351s
user	10m22.725s
sys	29m11.153s
pyrodigal-gv open

real	151m59.457s
user	132m47.984s
sys	361m4.035s
pyrodigal-gv closed

real	149m53.179s
user	130m49.922s
sys	355m32.864s
pyrodigal-gv closed start

real	153m39.169s
user	132m39.385s
sys	366m34.519s


Currently, `pyrodigal-rv` takes >5h to run compared to 12m for vanilla `pyrodigal` forcing gcode 1 or `pyrodigal-gv` with >2h.

To reduce runtime we can remove models that have not been used in the refseq benchmark if they are not important (ie. use genetic code 1 or another gc that is already well represented)

In [3]:
%%bash
cd refseq_genomes
# get used models
grep model predictions_closed.gff | cut -f 2 -d '|' | sort -u | grep -v Anaplasma > used_models
# get all models
grep description ~/data/software/pyrodigal-rv/pyrodigal_rv/meta.json | cut -f 2 -d '|' | sort -u > all_models
# compare
comm -3 used_models all_models

	Betaormycoviridae_1_model
	Botourmiaviridae_4_model
	Fiersviridae_16_model
	Fiersviridae_4_model
	Fiersviridae_6_model
	Mitoviridae_16_model
	Narnaviridae_16_model
	Narnaviridae_6_model
	Picobirnaviridae_5_model
	Picobirnaviridae_6_model
	Splipalmiviridae_1_model
	Tombusviridae_16_model


Remove:
- Betaormycoviridae_1_model (gc 1 well represented)
- Splipalmiviridae_1_model (gc 1 well represented)
- Fiersviridae_4_model (gc 4 represented by mitoviruses)
- Fiersviridae_16_model (little evidence they actually use gc 16, no complete genomes)
- Botourmiaviridae_4_model (1 seq can use gc 1, the other needs 4 but can be covered with Mito 4)
- Narnaviridae_16_model (little evidence they actually use gc 16)

Concatenated:
- Mitovirus 4 genera